In [1]:
import os
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

### Aliena (a)

Falando um bocadinho do processo que o nosso grupo seguiu para resolver esta primeira alinea, a nossa primeira preocupação foi a geração de chaves publica e privada. Começamos por escolher dois números primos "q" e "p", ambos têm de ser de ordem 10^100, e calculamos o "n" (multiplicando "p" por "q") e o totiente de euler. A seguir escolhemos um número inteiro aleatório "e" entre 1 e "n" que seja co-primo com o totiente de euler (podemos verificar esta condição usando a função que no indica o máximo divisor comum entre dois números, no caso de não ser 1 temos de procurar outro "e"). Tendo este número "e", calculamos "d" de forma que este seja o inverso multiplicativo de "e" em modulo "n". Tendo tudo isto calculado, a chave privada é o par (n, d) e a chave pública é o par (n, e).

Para cifrar uma mensagem damos uso à chave publica do peer ("n", "e") e fazemos o seguinte cálculo "mensagem ^ e % n". O resultado é mensagem cifrada. Para decifrar usamos a nossa chave privada e fazemos o cálculo "mensagem_cifrada ^ d % n" e o resultado devolve a mensagem decifrada.

Para finalizar vamos agora comentar o encapsulamento e desencapsulamento gerado pelo grupo. Para encapsular uma chave publica de forma a obter uma chave simétrica, em primeiro lugar, escolhemos um "m" aleatório tal que 1 < m < n, de seguida ciframos esse m com o método descrito a cima. Transformamos o m em bytes e passamos por um KDF à escolha (no nosso caso escolhemos o HKDF) para obter a chave simétrica. Para desencapsular deciframos o m cifrado, passamos a bytes e passamos pelo KDF dado no encapsulamento e obtemos a chave simétrica.

In [2]:
class KEM_RSA:
    def __init__(self, salt):
        self.private_key = None
        self.public_key  = None
        self.salt = salt

    
    # Metodos Privados
    def __encrypt(self, message, peer_public_key):
        n, e = peer_public_key
        # message ^ e % n
        return power_mod(message, e, n)

    def __decrypt(self, cipher_message):
        n, d = self.private_key
        # cipher_message ^ d % n
        return power_mod(cipher_message, d, n)


    # Metodos publicos
    def generate_keys(self, tam):
        # Dois primos q e p (tam deve ser no minimo 1024)
        q = random_prime(2 ^ (tam / 2) - 1, true, 2 ^ (tam / 2 - 1))
        p = random_prime(2 ^ (tam / 2) - 1, true, 2 ^ (tam / 2 - 1))
        
        # n é o produto de q e p
        n = p * q
        
        # Função do totiente de euler
        phi = (p - 1) * (q - 1)
        
        # Escolher um inteiro e tal que 1 < e < phi
        # e e phi têm de ser relativamente primos entre si
        # Para isso verificamos o máximo divisor comum entre os dois
        e = ZZ.random_element(phi)
        while gcd(e, phi) != 1:
            e = ZZ.random_element(phi)

        # Escolher d tal que d seja o inverso multiplicativo de e
        d = inverse_mod(e, phi)

        # Chaves
        self.private_key = (n, d)
        self.public_key  = (n, e)

    def encapsulation(self, peer_public_key, my_m):
        n, e = peer_public_key

        if (my_m == 0):
            # Encontrar m tal que 1 < m < n e depois encriptar
            m = ZZ.random_element(n)
        else:
            m = my_m
        m_encrypted = self.__encrypt(m, peer_public_key)

        # Obter chave
        m_in_bytes = int.to_bytes(int(m), int(m).bit_length(), "big")
        symetric_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=self.salt,
            info=None,
        ).derive(m_in_bytes)

        # Podemos mandar sem ser em bytes?        
        return symetric_key, m_encrypted

    def decapsulation(self, m_encrypted):
        m = self.__decrypt(m_encrypted)
        m_in_bytes = int.to_bytes(int(m), int(m).bit_length(), "big")
        symetric_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=self.salt,
            info=None,
        ).derive(m_in_bytes)

        return symetric_key

In [168]:
salt = os.urandom(12)

bob_RSA   = KEM_RSA(salt)
alice_RSA = KEM_RSA(salt)

# Gerar chaves
bob_RSA.generate_keys(1024)
alice_RSA.generate_keys(1024)


symetric_key_bob, m_encrypted = bob_RSA.encapsulation(alice_RSA.public_key, 0)
symetric_key_alice = alice_RSA.decapsulation(m_encrypted)

print("Chave do lado do bob:   ", symetric_key_bob)
print("Chave do lado da alice: ", symetric_key_alice)

Chave do lado do bob:    b'\xe1]\xe6\xb0\x06}\xe2\x9e\xee\x99\xb4\xf0\x00\xf4\x8bW\xcd\xde\xa2\xd2\x81u\x83"\x9eE>\x92\xde\xb8n\x9e'
Chave do lado da alice:  b'\xe1]\xe6\xb0\x06}\xe2\x9e\xee\x99\xb4\xf0\x00\xf4\x8bW\xcd\xde\xa2\xd2\x81u\x83"\x9eE>\x92\xde\xb8n\x9e'


### Alínea (b)

O primeiro passo para construir um PKE foi escolher duas funções hash, h(x) (sha256) e g(x) (BLAKE2s). Tendo escolhido estas duas funções, para cifrar uma mensagem começamos por aplicar a função h(x) mensagem obtendo assim a bit string "r". De seguida aplicamos a operação XOR à mensagem e à bit string obtida passando o valor "r" pela função hash g(x) (mensagem XOR h(g(mensgame))), o restultado desta operação devolve-nos o resultado de y. Somando as strings "y" e "r" obtemos "r'", convertemos "r'" a inteiro e agora vamos usar o KEM contruido anteriormente. Recordando que para realizar a operação de encapsulamento começamos por escolher um inteiro aleatório "m" entre 1 e "n", neste caso esse intereiro não precisará de ser calculado, visto que "m" terá o valor de "r'", obtendo assim um m cifrado e uma chave simetrica. Queremos finalmente calcular uma bit string "c" que resultará da operação XOR sobre a chave obtida do encapsulamento e o "r". O processo de cifragem deverá retornar o "y", o "c" e o "m cifrado". 

Para decifrar a mensagem pegamos no que foi retornado pelo processo de cifragem ("y", "c" e "m cifrado") e fazemos um processo análogo à cifragem. Desencapsulamos o "m cifrado" e obtemos uma bit string (que corresponde a uma chave simetrica) e calculamos um "r" com a operação XOR aplicada ao "c" e à chave simétrica. Somamos o "y" da cifragem a este novo "r" obtendo um novo "r'" que logo de seguida convertemos a um inteiro. Agora queremos fazer uma verificação para saber que os valores que obtemos sao de facto iguais aos que o nosso peer tem, para isso usamos o processo de encapsulamento criado acima usando a nossa chave publica e o "r'" inteiro como "m". Este resultado deverá resultar numa chave e num m que devem ser ter valores iguais aos obtidos anteriormente. Para obter a mensagem fazemos uma operação XOR sobre o "y" e o resultado do hash g(r).

In [173]:
class PKE:
    def __init__(self, kem):
        self.kem     = kem
    

    # Metodos Privados
    def __hash_functio_h(self, message):
        digest = hashes.Hash(hashes.SHA256())
        digest.update(message)
        return digest.finalize()
    
    def __hash_functio_g(self, message):
        digest = hashes.Hash(hashes.BLAKE2s(32))
        digest.update(message)
        return digest.finalize()


    # Metodos Publicos
    def encrypt(self, message, peer_public_key):
        # r <- h(message)
        r = self.__hash_functio_h(message)
        # y <- message XOR g(r)
        y = bytes([a ^^ b for a, b in zip(message, self.__hash_functio_g(r))])
        # Concatenamos y e r
        new_r = y + r
        new_r_int = int.from_bytes(new_r, "big") 

        # Vamos agora usar o KEM construido na alinea anterior
        symetric_key, m_encrypted = self.kem.encapsulation(peer_public_key, new_r_int)
        
        # c = symetric_key XOR r
        c = bytes([a ^^ b for a, b in zip(symetric_key, r)])

        return y, m_encrypted, c
        
    def decrypt(self, y, m_encrypted, c):
        # Usamos o KEM para obter a chave
        symetric_key = self.kem.decapsulation(m_encrypted)
        
        # Repetimos o processo mas com r <- c XOR symetric_key
        r = bytes([a ^^ b for a, b in zip(c, symetric_key)])
        new_r = y + r
        new_r_int = int.from_bytes(new_r, "big") 

        # Verificamos pois f(public_key, new_r) = (symetric_key, m_encrypted)
        new_symetric_key, new_m_encrypted = self.kem.encapsulation(self.kem.public_key, new_r_int)
        if symetric_key != new_symetric_key:
            print("Not YO: symetric_key is diferent")
        else:
            if m_encrypted != new_m_encrypted:
                print("Not YO: m_encrypted is diferent")
            else:
                message = bytes([a ^^ b for a, b in zip(y, self.__hash_functio_g(r))])
                print("Yo the message is: ", message)

In [174]:
salt = os.urandom(12)

bob_RSA = KEM_RSA(salt)
bob_RSA.generate_keys(1024)

alice_RSA = KEM_RSA(salt)
alice_RSA.generate_keys(1024)


bob_PKE = PKE(bob_RSA)
y, m_encrypted, c = bob_PKE.encrypt(b"This message is YO", alice_RSA.public_key)

alice_PKE = PKE(alice_RSA)
alice_PKE.decrypt(y, m_encrypted, c)

Yo the message is:  b'This message is YO'
